In [28]:
from torch.utils.data import Dataset
import os
import json
import torch
from transformers import AutoTokenizer
import numpy as np
import string
from nltk.corpus import stopwords
from nltk import word_tokenize,sent_tokenize
import nltk
from tqdm import tqdm
from datasets import load_from_disk,load_dataset
tok_b = AutoTokenizer.from_pretrained("/mnt/nas4/m11115088/WordRank/CNN_Model/GPT_final_chpt_bart")
tok_p = AutoTokenizer.from_pretrained("CNN_Model/GPT_final_chpt_pegasus")
# dataset = load_from_disk("/mnt/nas4/m11115088/WordRank/Dataset/cnndm_gpt_all")["test"]
dataset = load_from_disk("/mnt/nas4/m11115088/WordRank/Dataset/xsum/xsum_test_gpt4_turbo")["test"]
len_1_b = 0
len_greater_1_b = 0
len_1_p = 0
len_greater_1_p = 0
all_w = 0
keyword=[]
for data in tqdm(dataset):
    src_txt = data["document"]
    abstract = data["summary"]
    src_txt_list = word_tokenize(src_txt)
    src_txt_list_lower = word_tokenize(src_txt.lower())
    for abs_stn in sent_tokenize(abstract):
        nomal_abstract=word_tokenize(abs_stn)
        for i,(word,tag) in enumerate(nltk.pos_tag(nomal_abstract)):
            # if tag not in ["CC","CD","DT","MD","IN","VBZ","TO","WRB","WP$","WDT","WP",":"]:
            # if "NN" in tag or "VB" in tag or "JJ" in tag or "RB" in tag or "CD" in tag or "RP" in tag:
            if "NN" in tag or "VB" in tag or "JJ" in tag or "RB" in tag or "CD" in tag:
            # if "NN" in tag or "VB" in tag or "JJ" in tag or "RB" in tag:
            # if tag not in ["CC","CD","DT","MD","IN","VBZ","TO","WRB","WP$","WDT","WP",":"]:
            # if "NN" in tag or "VB" in tag or "JJ" in tag:
            # if "NN" in tag or "VB" in tag:
                word=word.strip(string.punctuation)
                if word !="" and  word not in keyword and (word.lower() not in stopwords.words('english')+list(string.punctuation)):
                    if word in src_txt_list:
                        keyword.append(word)
                    if word.lower() in src_txt_list:
                        keyword.append(word.lower())
                    if word.lower() in src_txt_list_lower:
                        w_idx = src_txt_list_lower.index(word.lower())
                        keyword.append(src_txt_list[w_idx])
keyword = list(set(keyword))
all_w+=len(keyword)
for word in keyword:
    t = tok_b.tokenize(word)
    if len(t)==1:
        len_1_b+=1
    else:
        len_greater_1_b+=1
    t = tok_p.tokenize(word)
    if len(t)==1:
        len_1_p+=1
    else:
        len_greater_1_p+=1
    # break

100%|██████████| 11334/11334 [01:30<00:00, 125.33it/s]


In [36]:
tok_b.tokenize("involvement")

['inv', 'olve', 'ment']

In [31]:
all_w,len_1_p,len_1_b,len_1_b/all_w,len_1_p/all_w

(22924, 15137, 3724, 0.837550165765137, 0.6603123364159833)